In [3]:
from pathlib import Path
import SimpleITK as sitk
import pydicom
import os
import pandas as pd

def read_dicom_metadata(filepath):
    try:
        return pydicom.read_file(filepath, stop_before_pixels=True)
    except Exception as e:
        print(f"Error reading DICOM metadata: {e}")
        return None

def sanitize_filename(filename):
    return filename.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(":", "").replace("-", "").replace("+", "")

def get_metadata(subpath):
    dicom_file = next(subpath.glob('*'), None)
    if dicom_file:
        img = read_dicom_metadata(dicom_file)
        if img:
            return {
                'name': getattr(img, 'SeriesDescription', "None"),
                'ID': getattr(img, 'PatientID', "None"),
                'imgID': getattr(img, 'AccessionNumber', "None"),
                'actime': getattr(img, 'AcquisitionTime', "None"),
                'series_number': getattr(img, 'SeriesNumber', "None"),
                'study_date': getattr(img, 'StudyDate', "None"),
                'manufacturer': getattr(img, 'Manufacturer', "None"),
                'slice_thickness': getattr(img, 'SliceThickness', "None"),
                'modality': getattr(img, 'Modality', "None"),
                'gender': getattr(img, 'PatientSex', "None"),
                'age': getattr(img, 'PatientAge', "None"),
                'height': getattr(img, 'PatientSize', "None"),
                'weight': getattr(img, 'PatientWeight', "None"),
                'file_count': len(list(subpath.glob('*')))
            }
    return {'name': "None", 'ID': "None", 'imgID': "None", 'actime': "None"}

def filter_uniform_size_series(filenames):
    sizes = {}
    for fn in filenames:
        try:
            img = pydicom.read_file(fn)
            size = img.pixel_array.shape
            sizes.setdefault(size, []).append(fn)
        except AttributeError as e:
            print(f"Warning: Unable to access pixel data for file {fn}: {e}")
        except Exception as e:
            print(f"Error processing file {fn}: {e}")
    max_size = max(sizes.keys(), key=lambda size: len(sizes[size]), default=None)
    return sizes.get(max_size, [])

def dcmRead(path):
    reader = sitk.ImageSeriesReader()
    filenamesDICOM = reader.GetGDCMSeriesFileNames(str(path))
    uniform_filenames = filter_uniform_size_series(filenamesDICOM)
    if uniform_filenames:
        reader.SetFileNames(uniform_filenames)
        try:
            img = reader.Execute()
            return img, sitk.GetArrayFromImage(img)
        except RuntimeError as e:
            print(f"Error reading DICOM series at {path}: {e}")
            return None, None
    else:
        print(f"No uniform series found at {path}")
    return None, None

def create_path(base, *paths):
    path = Path(base, *paths)
    path.mkdir(parents=True, exist_ok=True)
    return path

def sort_subfolders_by_actime(subfolders):
    subfolders_metadata = [(subfolder, get_metadata(subfolder)) for subfolder in subfolders]
    return sorted(subfolders_metadata, key=lambda x: x[1]['actime'])

def determine_and_create_save_path(NIIsavepath, seriesname_lower, MR_ID, imgID, ACtime):
    NIIsavepath = Path(NIIsavepath)
    clean_seriesname = sanitize_filename(seriesname_lower)
    savePath = NIIsavepath / f"{MR_ID}_{imgID}_{ACtime}_{clean_seriesname}.nii.gz"
    return savePath

def export_metadata_to_excel(metadata_list, excel_path):
    df = pd.DataFrame(metadata_list)
    df.to_excel(excel_path, index=False)
    print(f"Metadata exported to {excel_path}")

def series_extract(dcmpath, NIIsavepath, excel_savepath):
    dcmpath = Path(dcmpath)
    NIIsavepath = Path(NIIsavepath)
    
    metadata_list = []
    
    for filefolder in Path(dcmpath).iterdir():
        if filefolder.is_dir():
            sorted_subfolders = sort_subfolders_by_actime(filefolder.iterdir())
            for subfolder, metadata in sorted_subfolders:
                seriesname_lower = metadata['name'].lower()
                if metadata['file_count'] >= 5:  # Only process if there are 5 or more files
                    img, _ = dcmRead(subfolder)
                    if img:
                        savePath = determine_and_create_save_path(NIIsavepath, seriesname_lower, metadata['ID'], metadata['imgID'], metadata['actime'])
                        try:
                            sitk.WriteImage(img, str(savePath))
                            print(f"{metadata['ID']} transformation finished")
                        except Exception as e:
                            print(f"Error writing NIfTI file {savePath}: {e}")
                else:
                    print(f"Skipped processing folder {subfolder} due to insufficient DICOM files (<5).")
                
                metadata_list.append(metadata)  # Store metadata regardless of whether the series was processed
    
    export_metadata_to_excel(metadata_list, excel_savepath)

if __name__ == "__main__":
    dcmpath = "/Volumes/MR_analysis/MRCP_DICOM/"
    NIIsavepath = "/Volumes/MR_analysis/MRCP_NII/"
    excel_savepath = "/Volumes/MR_analysis/MRCP_metadata36.xlsx"
    os.makedirs(NIIsavepath,exist_ok=True)
    
    series_extract(dcmpath, NIIsavepath, excel_savepath)

ImageSeriesReader (0x105cb5830): Non uniform sampling or missing slices detected,  maximum nonuniformity:337.371



60010072483LB transformation finished


ImageSeriesReader (0x105ce0150): Non uniform sampling or missing slices detected,  maximum nonuniformity:543.784



60010072483LB transformation finished


ImageSeriesReader (0x105cb5830): Non uniform sampling or missing slices detected,  maximum nonuniformity:183.048



...

60016525837DXG transformation finished


ImageSeriesReader (0x105ce2a90): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000609396



60016525837DXG transformation finished


ImageSeriesReader (0x105cf7670): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000609396



60016525837DXG transformation finished
Metadata exported to /Volumes/MR_analysis/MRCP_metadata36.xlsx


In [ ]:
# 删除或移动一个文件夹下的所有文件
import os
import shutil

def del_file(path):
    ls = os.listdir(path)
    for i in ls:
        c_path = os.path.join(path, i)
        if os.path.isdir(c_path):  # 如果是文件夹那么递归调用一下
            del_file(c_path)
        else:  # 如果是一个文件那么直接删除
            os.remove(c_path)
    print('文件已经清空完成')
def move_file(path, path_moved):
    if not os.path.exists(path_moved):
        os.makedirs(path_moved)


    for item in os.listdir(path):
        src = os.path.join(path, item)
        dst = os.path.join(path_moved, item)


        if os.path.isdir(src):
            # If it's a directory, recursively move it
            move_file(src, dst)
        else:
            # If it's a file, check if it already exists in the destination
            if os.path.exists(dst):
                # If a file with the same name exists, replace it
                os.remove(dst)
            shutil.move(src, dst)

    if os.path.isdir(path):
        # If it's a directory, remove the source directory after all contents are moved
        os.rmdir(path)
    print('Directory and its contents have been moved with file replacement.')
del_file("/Volumes/ZILINGZHOU/DICOM/")
# move_file("/Users/ziling/Desktop/MRCP/MRCP_all_NII/","/Volumes/Extreme_zzl/MRCP_NII/")
# move_file("I:/CRC_DICOM_MRE/","H:/CRC_DICOM_MRE/")


# path = "L:/IBD_GIST_DICOM_origin/control/"
# savepath = "I:/IBD_GIST_DICOM_origin/control/"
# for file in os.listdir(path):
#     subpath = os.path.join(path,file)
#     move_file(subpath,os.path.join(savepath,file))